In [46]:
from utils import *
import chess as ch
import matplotlib.pyplot as plt

In [57]:

def put_text(text, image, position = (50, 50)):
    image = image.copy()
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_color = (255, 255, 255)  # Text color in BGR format
    line_type = 2

    # Add text to the image
    cv2.putText(image, text, position, font, font_scale, font_color, line_type)

    return image


In [47]:
def plot_chessboard(statistic):
    plt.imshow(statistic.reshape((8,8)), cmap='viridis', interpolation='nearest')
    plt.colorbar()  # Show color bar
    plt.title('Heatmap of 8x8 Data')
    plt.show()

In [2]:
chessboard = np.array((
    ['h8', 'h7', 'h6', 'h5', 'h4', 'h3', 'h2', 'h1'],
    ['g8', 'g7', 'g6', 'g5', 'g4', 'g3', 'g2', 'g1'],
    ['f8', 'f7', 'f6', 'f5', 'f4', 'f3', 'f2', 'f1'],
    ['e8', 'e7', 'e6', 'e5', 'e4', 'e3', 'e2', 'e1'],
    ['d8', 'd7', 'd6', 'd5', 'd4', 'd3', 'd2', 'd1'],
    ['c8', 'c7', 'c6', 'c5', 'c4', 'c3', 'c2', 'c1'],
    ['b8', 'b7', 'b6', 'b5', 'b4', 'b3', 'b2', 'b1'],
    ['a8', 'a7', 'a6', 'a5', 'a4', 'a3', 'a2', 'a1'],
))

chessboard = chessboard.reshape(64)

### Easy1

In [14]:
circles_count = np.load('circles_centers/Easy1.npy')
used_frames = np.load('used_frames/Easy1.npy')
circles_count.shape

(1484, 64)

In [53]:
board = ch.Board()

made_moves = list()

for i in range(0, circles_count.shape[0]-3, 3):
    zeros1 = np.median(circles_count[i:i+3], axis=0) == 0
    zeros2 = np.median(circles_count[i+3:i+6], axis=0) == 0

    piece_positions = np.array([bool(board.piece_at(square)) for square in ch.SQUARES])
    piece_positions = piece_positions.reshape((8,8)).T
    piece_positions = np.rot90(piece_positions, k=2)
    piece_positions = piece_positions.reshape(64)

    undetected_figures = piece_positions & zeros1 & zeros2

    common = zeros1 & zeros2
    zeros1[common] = False
    zeros2[common] = False 

    zeros2[undetected_figures] = True

    if np.sum(zeros1) > 0 and np.sum(zeros2) > 0:
        start_positions = chessboard[zeros2]
        end_positions = chessboard[zeros1]


        possible_moves = list()
        for position1 in start_positions:
            for position2 in end_positions:
                move = position1 + position2
                if ch.Move.from_uci(move) in board.legal_moves:
                    possible_moves.append(move)


        if len(possible_moves) > 0:
            move = possible_moves[-1]
            print(move)
            print(possible_moves)
            board.push(ch.Move.from_uci(move))
            print(board)
            made_moves.append((move, i))      
        
                    

d2d4
['d2d4']
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N B Q K B N R
b7b5
['b7b5']
r n b q k b n r
p . p p p p p p
. . . . . . . .
. p . . . . . .
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N B Q K B N R
a2a4
['a2a4']
r n b q k b n r
p . p p p p p p
. . . . . . . .
. p . . . . . .
P . . P . . . .
. . . . . . . .
. P P . P P P P
R N B Q K B N R
c8b7
['c8b7']
r n . q k b n r
p b p p p p p p
. . . . . . . .
. p . . . . . .
P . . P . . . .
. . . . . . . .
. P P . P P P P
R N B Q K B N R
b1c3
['c2c3', 'b1c3']
r n . q k b n r
p b p p p p p p
. . . . . . . .
. p . . . . . .
P . . P . . . .
. . N . . . . .
. P P . P P P P
R . B Q K B N R
a7a6
['b8a6', 'b7a6', 'a7a6']
r n . q k b n r
. b p p p p p p
p . . . . . . .
. p . . . . . .
P . . P . . . .
. . N . . . . .
. P P . P P P P
R . B Q K B N R
c1d2
['e1d2', 'c1d2']
r n . q k b n r
. b p p p p p p
p . . . . . . .
. p . . . . . .
P . . P . . . .
. . N . . . . .
. P P 

In [39]:
chess = video_from_path('recordings/Easy1.mp4')

width = int(chess.get(3))
height = int(chess.get(4))

fps = chess.get(cv2.CAP_PROP_FPS)

writer = cv2.VideoWriter(
    f"test/comments.mp4",  # Updated filename
    cv2.VideoWriter_fourcc(*"mp4v"),  # Codec for MP4 format (codec might vary)
    25,
    (width, height),
    True
)

i = 0
j = 0
k = 0
text = 'no move done'
next_move = made_moves[0]
while chess.isOpened():
    ret, frame = chess.read()
    if ret:
        if next_move[1] == i:
            text = next_move[0]
            k += 1
            if k < len(made_moves):
                next_move = made_moves[k]

        if used_frames[j]: 
            i += 1
        if j == 1620:
                    break
        image = put_text(text, frame)
        writer.write(image)
        j += 1
        
    else:
        break

writer.release()

Video loaded
912 720
30.00030000300003


### Easy2

In [40]:
circles_count = np.load('circles_centers/Easy2.npy')
used_frames = np.load('used_frames/Easy2.npy')
circles_count.shape

(1687, 64)

In [42]:
board = ch.Board()

made_moves = list()

for i in range(0, circles_count.shape[0]-3, 3):
    zeros1 = np.median(circles_count[i:i+3], axis=0) == 0
    zeros2 = np.median(circles_count[i+3:i+6], axis=0) == 0

    piece_positions = np.array([bool(board.piece_at(square)) for square in ch.SQUARES])
    piece_positions = piece_positions.reshape((8,8)).T
    piece_positions = np.rot90(piece_positions, k=2)
    piece_positions = piece_positions.reshape(64)

    undetected_figures = piece_positions & zeros1 & zeros2

    common = zeros1 & zeros2
    zeros1[common] = False
    zeros2[common] = False 

    zeros2[undetected_figures] = True

    if np.sum(zeros1) > 0 and np.sum(zeros2) > 0:
        start_positions = chessboard[zeros2]
        end_positions = chessboard[zeros1]


        possible_moves = list()
        for position1 in start_positions:
            for position2 in end_positions:
                move = position1 + position2
                if ch.Move.from_uci(move) in board.legal_moves:
                    possible_moves.append(move)


        if len(possible_moves) > 0:
            move = possible_moves[-1]
            print(move, i)
            print(possible_moves)
            board.push(ch.Move.from_uci(move))
            print(board)
            made_moves.append((move, i))        

d2d4 63
['d2d4']
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N B Q K B N R
b7b5 276
['b7b5']
r n b q k b n r
p . p p p p p p
. . . . . . . .
. p . . . . . .
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N B Q K B N R
a2a4 438
['a2a4']
r n b q k b n r
p . p p p p p p
. . . . . . . .
. p . . . . . .
P . . P . . . .
. . . . . . . .
. P P . P P P P
R N B Q K B N R
c8b7 849
['c8b7']
r n . q k b n r
p b p p p p p p
. . . . . . . .
. p . . . . . .
P . . P . . . .
. . . . . . . .
. P P . P P P P
R N B Q K B N R
b1c3 876
['c2c3', 'b1c3']
r n . q k b n r
p b p p p p p p
. . . . . . . .
. p . . . . . .
P . . P . . . .
. . N . . . . .
. P P . P P P P
R . B Q K B N R
a7a6 930
['b8a6', 'b7a6', 'a7a6']
r n . q k b n r
. b p p p p p p
p . . . . . . .
. p . . . . . .
P . . P . . . .
. . N . . . . .
. P P . P P P P
R . B Q K B N R
c1d2 1386
['e1d2', 'c1d2']
r n . q k b n r
. b p p p p p p
p . . . . . . .
. p . . . . . .
P . . P . 

In [43]:
chess = video_from_path('circles_detected/Easy2.mp4')

width = int(chess.get(3))
height = int(chess.get(4))

fps = chess.get(cv2.CAP_PROP_FPS)

writer = cv2.VideoWriter(
    f"test/test_frame_count.mp4",  # Updated filename
    cv2.VideoWriter_fourcc(*"mp4v"),  # Codec for MP4 format (codec might vary)
    fps,
    (width, height),
    True
)

j = 0
while chess.isOpened():
    ret, frame = chess.read()
    if ret:
        image = put_text(str(j), frame)
        writer.write(image)
        j += 1
        
    else:
        break

writer.release()

Video loaded
920 720
30.0
